In [1]:
from mysql_connection import get_cursor
import time
import pandas as pd

cursor = get_cursor()
# Start the timer
start = time.perf_counter()

## Call get food checs by id with the user id

In [2]:
user_id = (1,)
foodchecks = cursor.callproc("get_foodchecks_by_id", user_id)
checks = []
# stored_results is iterable, have to do this way
for result in cursor.stored_results():
    checks = result.fetchall()[0][1:]

print(checks)

(0, 0, 0, 0, 1, 1, 1, 1, 1)


## Parse all the results, and create the WHERE clause

In [3]:
column_rows = ["IsVegetarian", "IsVegan", "IsHala", "IsKosher", "HasLactose", "HasNuts", "HasGluten", "HasEgg", "HasSoy"]

def print_food_checks(result) :
    for i in range(len(result)):
        print(f"{column_rows[i]} : {result[i]}")

print_food_checks(checks)

def food_checks_builder(result):
    food_checks = []
    for i in range(len(result)):
        if(result[i]):
            food_checks.append(f"{column_rows[i]}=1")
    return " AND ".join(food_checks)

print("---------------------------------")
print(food_checks_builder(checks))

IsVegetarian : 0
IsVegan : 0
IsHala : 0
IsKosher : 0
HasLactose : 1
HasNuts : 1
HasGluten : 1
HasEgg : 1
HasSoy : 1
---------------------------------
HasLactose=1 AND HasNuts=1 AND HasGluten=1 AND HasEgg=1 AND HasSoy=1


## Get all the user id's with at least the food checks that you have
### i.e. You are vegetarian... Some one who is vegetarian, egg-free, and lactose-free will also be selected 

In [4]:
# UserID, FoodCheckID, IsVegetarian, IsVegan, IsHala, IsKosher, HasLactose, HasNuts, HasGluten, HasEgg, HasSoy
cursor.execute(f"SELECT UserID FROM tat.UserFoodCheckView WHERE {food_checks_builder(checks)};")
# Result contains an array of all users that match ATLEAST the food tags our user has
result = cursor.fetchall()
print(result[0])

(1,)


In [5]:
ids = "("
for i in result:
    ids += str(i[0]) + ", "
ids = ids[:-2]
ids += ")"
cursor.execute(f"SELECT * FROM FoodOpinion WHERE UserID IN{ids};")
result = cursor.fetchall()
print(result)
data = []
# Loop through and calc the % of total swipes that are right
for i in range(len(result)):
    right, left = result[i][2], result[i][3]
    total = right + left
    percentage = float(right) / float(total) 
    # We want the data in a range between 0 - 2, hence * by 2, rounded to 1 dp
    value = round(percentage * 2, 1)
    data.append([result[i][0], result[i][1], result[i][2], result[i][3], value ])


[(1, 1, 10, 2, 0, 0), (1, 2, 15, 2, 0, 0), (1, 3, 0, 2, 0, 0), (2, 1, 10, 2, 0, 0), (3, 1, 10, 2, 0, 0)]


## Normalise the right / left swipes to a rating between 0 - 2

In [6]:
df = pd.DataFrame(data)

# Only using Right and Left columns to make it easier to view data...
# In the actual app, we will only have userid and right swipe percentage
df.columns = ["UserID", "Food Tag ID", "Right", "Left", "Right swipe percentage"]
print(df)

   UserID  Food Tag ID  Right  Left  Right swipe percentage
0       1            1     10     2                     1.7
1       1            2     15     2                     1.8
2       1            3      0     2                     0.0
3       2            1     10     2                     1.7
4       3            1     10     2                     1.7


## Time the exectution

In [7]:
end = time.perf_counter()
print(f"Done in {end - start:0.4f} seconds")

Done in 0.1794 seconds
